<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_6_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'fatigue',
 'rt',
 'loss',
 'by',
 'sodium',
 'limb',
 'cramp',
 'weakness',
 'at',
 'disease',
 'that',
 'stomach',
 'or',
 'as',
 'depression',
 'back',
 '-',
 'patients',
 'i',
 'from',
 'this',
 'cramps',
 'not',
 'upper',
 'joint',
 'gas',
 'memory',
 'drug',
 'leg',
 'mg',
 'you',
 'all',
 'legs',
 'pains',
 'headache',
 'be',
 'knee',
 'are',
 'group',
 'shoulder',
 'study',
 'unable',
 'arthralgia',
 'treatment',
 'gastrointestinal',
 'neck',
 'excessive',
 'foot',
 'an',
 'after',
 'it',
 'aches',
 'arthritis',
 'hip',
 'amp',
 'have',
 'numbness',
 'voltaren',
 'like',
 'arthrotec',
 'insomnia',
 'p',
 'no',
 'nausea',
 'release',
 'increased',
 '/',
 'both',
 'gluten',
 'asthenia',
 'can',
 'bleeding',
 'abdomi

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.842548,3.476300,0.443353,1:36:58


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.531582,3.415551,0.448982,1:36:31


In [25]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.062358,3.468951,0.362299,03:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.913022,2.812383,0.495989,03:37


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.286166,2.551876,0.556150,07:27


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.040384,2.212602,0.597594,07:40
1,1.745143,2.037211,0.627005,08:02


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.529092,1.832263,0.668449,08:39
1,1.281533,1.749276,0.687166,07:40


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.274808,1.593790,0.719251,08:27
1,1.126173,1.512331,0.720588,08:21


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.024038,1.422937,0.744652,08:05
1,0.862617,1.352075,0.747326,07:47


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.830202,1.296270,0.756684,08:32
1,0.718441,1.287168,0.770053,08:21


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.692037,1.256032,0.762032,07:46
1,0.687792,1.209238,0.779412,07:34
2,0.590637,1.131384,0.790107,08:27
3,0.487972,1.109286,0.792781,08:01


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.497157,1.186892,0.788770,08:12
1,0.544029,1.154180,0.782086,08:00
2,0.440282,1.096287,0.796791,08:11
3,0.383530,1.095886,0.795455,08:17


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.386636,1.153383,0.800802,08:18
1,0.432745,1.094055,0.806150,08:12
2,0.375910,1.031955,0.808824,08:45
3,0.343916,1.034638,0.811497,08:03


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.306914,1.113363,0.812834,08:09
1,0.339078,1.120398,0.814171,08:25
2,0.306428,1.129953,0.811497,08:12
3,0.266552,1.081938,0.816845,08:38


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.263340,1.085434,0.814171,08:23
1,0.294137,1.092252,0.818182,08:02
2,0.277417,1.186075,0.812834,07:48
3,0.229061,1.136785,0.818182,08:21


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.225417,1.163229,0.816845,07:50
1,0.265191,1.136444,0.824866,07:50
2,0.252134,1.083742,0.830214,08:40
3,0.206674,1.124961,0.831551,08:11


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.207133,1.206470,0.827540,07:43
1,0.231433,1.164257,0.827540,08:48
2,0.227131,1.040035,0.831551,08:37
3,0.175037,1.133764,0.832888,08:48


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.219837,1.142723,0.828877,08:13
1,0.204853,1.087701,0.838235,08:20


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.200657,1.123935,0.835562,08:21
1,0.173716,1.058166,0.844920,08:37


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.190879,1.121562,0.840909,08:10


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold6')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 5, 8, 1026, 8, 8, 8, 8, 8, 8, 8, 102, 143, 27, 31, 34, 37, 37, 37, 37, 182, 37, 37, 37, 37, 37, 402, 41, 41, 41, 76, 49, 183, 100, 49, 100, 100, 100, 100, 100, 51, 53, 53, 53, 53, 53, 998, 767, 58, 58, 577, 422, 64, 67, 71, 71, 289, 76, 864, 522, 228, 624, 624, 624, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 182, 289, 182, 624, 624, 624, 79, 80, 80, 80, 80, 80, 80, 81, 662, 83, 89, 90, 91, 91, 91, 91, 91, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 96, 96, 100, 545, 100, 100, 100, 100, 185, 316, 108, 108, 108, 108, 864, 462, 111, 111, 111, 112, 255, 114, 330, 117, 117, 118, 453, 76, 469, 626, 125, 35, 125, 125, 125, 125, 125, 125, 778, 210, 741, 319, 135, 89, 246, 139, 151, 847, 925, 157, 157, 163, 178, 171, 172, 172, 172, 172, 172, 173, 607, 179, 182, 182, 182, 183, 183, 183, 183, 184, 807, 187, 187, 187, 189, 190, 190, 191, 34, 197, 202, 205, 210, 210, 289, 190, 214, 214, 214, 214, 214,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

631
0.7286374133949192
